In [ ]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn import preprocessing

### 2015 ~ 2020년도 데이터 가져오기

In [ ]:
filenames=['/content/drive/MyDrive/teamproject/data/baseball_' + str(x) + '.csv' for x in list(range(2015,2021))]
filenames

In [ ]:
data = pd.DataFrame()
for filename in tqdm(filenames):
    temp = pd.read_csv(filename)
    data = pd.concat([data,temp])

In [ ]:
baseball_data = data.copy()

### 날짜 데이터, HEADER_NO 삭제

In [ ]:
baseball_data = baseball_data.drop(['G_ID','GDAY_DS','HEADER_NO'], axis=1)
baseball_data.info()

### 무승부 제거

In [ ]:
print(baseball_data['win'].value_counts())
baseball_data = baseball_data[baseball_data['win']!=0.5]
print(baseball_data['win'].value_counts())

### 팀 명 라벨 인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(baseball_data['T_ID'])
baseball_data.loc[:,'T_ID'] = le.transform(baseball_data['T_ID'])
baseball_data.loc[:,'VS_T_ID'] = le.transform(baseball_data['VS_T_ID'])
baseball_data.head()

### 데이터 형태가 object인 값(TB_SC)을 라벨 인코딩

In [ ]:
le = LabelEncoder()
le.fit(baseball_data['TB_SC'])
baseball_data.loc[:,'TB_SC'] = le.transform(baseball_data['TB_SC'])
baseball_data.head()

### 결정 트리를 이용하여 주요 변수 알아내기
- 결정 트리의 경우 별도의 전처리가 필요 없음
- train_data1, train_target1

In [ ]:
train_data1 = baseball_data[['T_ID', 'VS_T_ID', 'TB_SC', 'PA', 'AB', 'RBI', 'RUN', 'HIT', 'H2', 'H3',
       'HR', 'SB', 'CS', 'SF', 'BB', 'HP', 'KK', 'GD', 'LOB', 'P_HRA_RT',
       'P_AB_CN', 'P_HIT_CN', 'OBP', 'OOO']]
train_target1 = baseball_data['win']

### 데이터 세트 나누기

In [ ]:
from sklearn.tree import DecisionTreeClassifier

x_train, x_test, y_train, y_test = train_test_split(train_data1, train_target1, test_size=0.2, random_state=42)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

### 결정 트리 모델 학습

In [ ]:
dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(x_train,y_train)

### 결정 트리 모델 시각화

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(dt_clf, out_file='tree.dot',class_names=['패','승','무'], feature_names = x_train.columns, impurity=True, filled=True)

In [ ]:
import graphviz

with open('tree.dot') as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

### 피처별 중요도

In [ ]:
import seaborn as sns
import numpy as np

print('Feature importances:\n{0}'.format(np.round(dt_clf.feature_importances_,3)))

for name, value in zip(x_train.columns, dt_clf.feature_importances_):
    print('{0} : {1:.3f}'.format(name, value))

sns.barplot(x=dt_clf.feature_importances_, y = x_train.columns)

In [ ]:
importances_series = pd.Series(dt_clf.feature_importances_,x_train.columns)

In [ ]:
importances_series.sort_values(ascending=False)

### 상관 관계(corr) 활용

In [ ]:
baseball_data.info()

In [ ]:
baseball_data_corr = baseball_data.corr()

In [ ]:
plt.figure(figsize=(15,6))
sns.heatmap(baseball_data_corr, cmap='Purples')

In [ ]:
baseball_data_corr['win'].sort_values(ascending=False)

### 무승부 제외 시, 주요 피처 뽑기(5개)
1. DecisionTree를 활용한 주요 피처
- RUN : 0.360827
- AB : 0.104176
- OOO: 0.060846
- PA : 0.047024
- OBP : 0.044471
2. 상관 관계(corr)를 활용한 주요 피처
- RUN : 0.559666
- PBI : 0.548687
- OBP : 0.508976
- OOO : 0.480763
- P_HIT_CN : 0.468711      
=> RUN, OOO, T_ID, VS_T_ID 으로 학습

In [ ]:
baseball_data[['RUN', 'OOO']].describe()

## 1. DecisionTree 학습

In [ ]:
baseball_data.info()

In [ ]:
DecisionTree_data = baseball_data[['T_ID','VS_T_ID','RUN','OOO']]
DecisionTree_target= baseball_data['win']

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(DecisionTree_data, DecisionTree_target, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

### GridSearchCV로 하이퍼파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

params={
    'max_depth':[4,8,12,16,20,24],
    'min_samples_leaf':[1,3,6,9,12,15],
    'min_samples_split':[8,16,20,24,28,32,36,40]
}

dt_clf = DecisionTreeClassifier(random_state=42)
grid_cv=GridSearchCV(dt_clf, param_grid=params,cv=5,n_jobs=-1)
grid_cv.fit(train_x,train_y)

In [ ]:
print('최적 하이퍼 파라미터:\n',grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

### 정확도 측정

In [ ]:
from sklearn.metrics import accuracy_score

dt_model = grid_cv.best_estimator_
pred = dt_model.predict(test_x)
print('예측 정확도: {0:.4f}'.format(accuracy_score(test_y, pred)))

### 2.RandomForest 학습

In [ ]:
RandomForest_data = baseball_data[['T_ID','VS_T_ID','RUN','OOO']]
RandomForest_target= baseball_data['win']

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(RandomForest_data, RandomForest_target, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

### GridSearchCV로 하이퍼파라미터 튜닝

In [ ]:
from sklearn.ensemble import RandomForestClassifier

params={
    'max_depth':[4,8,12,16,20,24],
    'min_samples_leaf':[1,3,6,9,12,15],
    'min_samples_split':[8,16,20,24,28,32,36,40]
}

rfc = RandomForestClassifier(n_estimators=50,random_state=42,n_jobs=-1)
grid_cv=GridSearchCV(rfc, param_grid=params,cv=5,n_jobs=-1)
grid_cv.fit(train_x,train_y)

### 정확도 측정

In [ ]:
from sklearn.metrics import accuracy_score

rfc_model = grid_cv.best_estimator_
pred = rfc_model.predict(test_x)
print('예측 정확도: {0:.4f}'.format(accuracy_score(test_y, pred)))

=> RUN, OOO, T_ID, VS_T_ID 으로 학습 시, 기존(RUN,AB,PA,OOO,T_ID,VS_T_ID)보다 정확도가 떨어짐